# INFORMATION TO USER
The notebook is used to generate the results at the following GitHub folders via the **Google Cloud Computing Platform**:

- https://github.com/oliver29063/MalariaDiagnosis/tree/master/BatchSize
- https://github.com/oliver29063/MalariaDiagnosis/tree/master/Optimizers

These results are also referred to in **Figure 4** of the manuscript in the **Results** subsection titled **Fine-Tuning Training Hyperparameters**. In short, this notebook is used to determine the ideal optimizer type, learning rate, and batch size.

Please note that this notebook requires about 24 GB of RAM to run properly, which exceeds Google Colab capacity and most personal laptops. Consequently, please adjust the number of images used to reduce RAM is necessary when testing the functionality of this notebook. 



# Package Versions
Standard Libraries for Python 3.6.9
- os
- shutil
- zipfile
- sys
- csv

Imported Libraries
- numpy: 1.18.5
- tensorflow: 2.2.0
- keras: 2.3.1
- sklearn: 0.22.2.post1
- cv2: 4.1.2
- PIL: 7.0.0

### Download NIH Malaria Dataset
For more information about the dataset, visit https://lhncbc.nlm.nih.gov/publication/pub9932

In [ ]:
# Import relevant packages
import numpy as np
import os
from shutil import copyfile
from zipfile import ZipFile

# Download NIH dataset zip file
!wget -nc ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip

# Extract images if not already extracted
ROOT_DIR = os.path.join("/", "content")
if not os.path.isdir("cell_images"):
    print("Extracting images...")
    with ZipFile(os.path.join("cell_images.zip"), "r") as zipObj:
        zipObj.extractall()
    print("Done!")

### Specify Number of Images to Use
In the manuscript we used 10000 images instead of the full set of approximately 22000 images to reduce computational burden. To check if the code runs properly on your device, we first recommend setting ```numImg``` equal to some small amount, such as 1000 to prevent RAM overload. 

In [2]:
numImg = 1000
numClass = numImg//2

### Basic Image Preprocessing
Here we just load our image data into two NumPy arrays ```Parasitized``` and ```Uninfected```, which each contain our set of 128x128 RGB images in a 4D NumPy array for each class.

In [ ]:
# Install and import relevant packages
import numpy as np
import os
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libxrender1
import cv2
from PIL import Image

# Create new folders to save rescaled images
if not os.path.isdir("RescaledSet"):
    os.mkdir("RescaledSet")
if not os.path.isdir("RescaledSet/Parasitized"):
    os.mkdir("RescaledSet/Parasitized")
if not os.path.isdir("RescaledSet/Uninfected"):
    os.mkdir("RescaledSet/Uninfected")

# Generate list of parasitized file names
ParasitizedFiles = os.listdir("cell_images/Parasitized/")
UninfectedFiles = os.listdir("cell_images/Uninfected/")

# Remove Thumb.db files
while 'Thumbs.db' in ParasitizedFiles: ParasitizedFiles.remove('Thumbs.db')   
while 'Thumbs.db' in UninfectedFiles: UninfectedFiles.remove('Thumbs.db')  

# Pre-allocate memory space for images
Parasitized = np.empty([numClass,128,128,3])
Uninfected = np.empty([numClass,128,128,3])

# Resize and load parasitized images
for i in range(numClass):
    TempImage = cv2.imread('cell_images/Parasitized/'+ParasitizedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Parasitized[i,:,:,:] = ResizedImage

# Resize and load uninfected images
for i in range(numClass):
    TempImage = cv2.imread('cell_images/Uninfected/'+UninfectedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Uninfected[i,:,:,:] = ResizedImage
    
print('Uninfected Dataset size is:',np.shape(Uninfected))
print('Parasitized Dataset size is:',np.shape(Parasitized))

### Create Model Inputs and Cross-Validation Groups
Now we create our ```Labels``` as our ground truth and merge our two image classes into a single NumPy array ```Dataset```. We then randomize the order of the images and create our five cross-validation set indices. 

In [4]:
# Generate dataset labels
ParasitizedLabels = np.repeat([[0,1]], numClass, axis=0)
UninfectedLabels = np.repeat([[1,0]], numClass, axis=0)
Labels = np.concatenate((ParasitizedLabels,UninfectedLabels), axis=0)

# Generate image dataset
Dataset = np.concatenate((Parasitized, Uninfected), axis=0)

# Generate 5-fold cross-validation groups
Spaces = np.linspace(0,numImg,6).astype('int')
CVIndices = np.random.permutation(Dataset.shape[0])
Index1, Index2, Index3, Index4, Index5 = CVIndices[:Spaces[1]], CVIndices[Spaces[1]:Spaces[2]], CVIndices[Spaces[2]:Spaces[3]], CVIndices[Spaces[3]:Spaces[4]], CVIndices[Spaces[4]:]

### Load VGG16 Model
Here we load our pretrained VGG16 model from Keras.

In [5]:
# Import VGG16 pre-trained neural network
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


### Import Relevant Packages for Neural Network Training

In [ ]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

!pip install scikit-learn
import sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

### List Learning Rates
Here we just create a list of each learning rate we wish to test.

In [7]:
# List of hyperparameters
LR = [1e-6, 1e-5, 1e-4]

### Test Different Learning Rates for Each Optimizer
Note that you must manually change the file names for each optimizer. The code chunk below only runs a series of learning rates for ONE specific optimizer. To test a different optimizer, the code must be manually commented out/in to pick the right optimizer.

In [ ]:
for k in LR:
    # Create empty lists to store results
    TrainLoss = []
    TrainAcc = []
    TestLoss = []
    TestAcc = []
    All_FPR = []
    All_TPR = []
    All_thresholds = []
    All_AUC = []

    for i in range(5):

        # Create the appropriate training and testing sets
        if i == 0:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index4,:]), axis=0)
            TestImages = Dataset[Index5,:]
            TestLabels = Labels[Index5,:]
        elif i == 1:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index4,:]
            TestLabels = Labels[Index4,:]
        elif i == 2:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index3,:]
            TestLabels = Labels[Index3,:]
        elif i == 3:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index2,:]
            TestLabels = Labels[Index2,:]
        else:
            TrainImages = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index2,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index1,:]
            TestLabels = Labels[Index1,:]

        # Pick optimizer type
        #opt = optimizers.SGD(lr=k, momentum=0.0, nesterov=True) # SGD w/ Nesterov
        #opt = optimizers.Nadam(lr=k, beta_1=0.9, beta_2=0.999)  # Nadam
        #opt = optimizers.RMSprop(lr=k,rho=0.9,momentum=0.0)     # RMSProp
        #opt = optimizers.Adamax(lr=k, beta_1=0.9, beta_2=0.999) # AdaMax
        opt = optimizers.Adam(lr=k, beta_1=0.9, beta_2=0.999)    # Adam

        base_model = VGG16(weights = "imagenet", include_top=False, input_shape = (128,128,3))

        x = base_model.output
        x = Flatten()(x)
        x = Dense(1024, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(1024, activation='relu')(x)
        x = Dropout(0.5)(x)
        predictions = Dense(2, activation="softmax")(x)
        model = Model(input = base_model.input, output = predictions)
        model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])

        # Train model and evaluate performance
        print('We are now training cross-validation set #',i+1)
        Results = model.fit(TrainImages, TrainLabels, epochs=40, batch_size=64, validation_data=(TestImages,TestLabels), validation_freq=1)

        # Display and store performance results
        Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
        Results.history['accuracy'] = [round(l, 4) for l in Results.history['accuracy']]
        Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]
        Results.history['val_accuracy'] = [round(l, 4) for l in Results.history['val_accuracy']]

        print('Training Loss:',Results.history['loss'])
        print('Training Accuracy:',Results.history['accuracy'])
        print('Validation Loss:',Results.history['val_loss'])
        print('Validation Accuracy:',Results.history['val_accuracy'])

        TrainLoss.append(Results.history['loss'])
        TrainAcc.append(Results.history['accuracy'])
        TestLoss.append(Results.history['val_loss'])
        TestAcc.append(Results.history['val_accuracy'])
        print('')


        # Predict values for test set
        Probabilities = model.predict(TestImages)

        # Calculate data for ROC curve
        FPR, TPR, thresholds = roc_curve(TestLabels[:,1], Probabilities[:,1])
        All_FPR.append(FPR)
        All_TPR.append(TPR)
        All_thresholds.append(thresholds)

    # Save and export as CSV files (must manually change file names for each optimizer type)
    with open("Adam_"+str(k)+"_TrainLoss.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(TrainLoss)
    with open("Adam_"+str(k)+"_TrainAcc.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(TrainAcc)
    with open("Adam_"+str(k)+"_TestLoss.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(TestLoss)
    with open("Adam_"+str(k)+"_TestAcc.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(TestAcc)
    with open("Adam_"+str(k)+"_FPR.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(All_FPR)
    with open("Adam_"+str(k)+"_TPR.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(All_TPR)
    with open("Adam_"+str(k)+"_Thresholds.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(All_thresholds)

### List Different Batch Sizes

In [10]:
Batch = [4,8,16,32,64,128,256]

### Test Different Batch Sizes


In [ ]:
for k in Batch:

    # Create empty lists to store results
    TrainLoss = []
    TrainAcc = []
    TestLoss = []
    TestAcc = []
    All_FPR = []
    All_TPR = []
    All_thresholds = []
    All_AUC = []

    for i in range(5):

        # Create the appropriate training and testing sets
        if i == 0:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index4,:]), axis=0)
            TestImages = Dataset[Index5,:]
            TestLabels = Labels[Index5,:]
        elif i == 1:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index4,:]
            TestLabels = Labels[Index4,:]
        elif i == 2:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index3,:]
            TestLabels = Labels[Index3,:]
        elif i == 3:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index2,:]
            TestLabels = Labels[Index2,:]
        else:
            TrainImages = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index2,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index1,:]
            TestLabels = Labels[Index1,:]

        base_model = VGG16(weights = "imagenet", include_top=False, input_shape = (128,128,3))

        x = base_model.output
        x = Flatten()(x)
        x = Dense(1024, activation="relu")(x)
        x = Dropout(0.5)(x)
        x = Dense(1024, activation="relu")(x)
        x = Dropout(0.5)(x)
        predictions = Dense(2, activation="softmax")(x)
        model = Model(input = base_model.input, output = predictions)
        sgd = optimizers.SGD(lr=1e-5, momentum=0.0, nesterov=True)
        model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])

        # Train model and evaluate performance (epoch set equal to 10 for speed, true experiment had epoch set to 75)
        print('We are now training cross-validation set #',i+1)
        Results = model.fit(TrainImages, TrainLabels, epochs=10, batch_size=k, validation_data=(TestImages,TestLabels), validation_freq=1)

        # Display and store performance results
        Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
        Results.history['accuracy'] = [round(l, 4) for l in Results.history['accuracy']]
        Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]
        Results.history['val_accuracy'] = [round(l, 4) for l in Results.history['val_accuracy']]

        print('Training Loss:',Results.history['loss'])
        print('Training Accuracy:',Results.history['accuracy'])
        print('Validation Loss:',Results.history['val_loss'])
        print('Validation Accuracy:',Results.history['val_accuracy'])

        TrainLoss.append(Results.history['loss'])
        TrainAcc.append(Results.history['accuracy'])
        TestLoss.append(Results.history['val_loss'])
        TestAcc.append(Results.history['val_accuracy'])
        print('')


        # Predict values for test set
        Probabilities = model.predict(TestImages)

        # Calculate data for ROC curve
        FPR, TPR, thresholds = roc_curve(TestLabels[:,1], Probabilities[:,1])
        All_FPR.append(FPR)
        All_TPR.append(TPR)
        All_thresholds.append(thresholds)

    # Save and export as CSV files
    with open(str(k)+"_TrainLoss.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(TrainLoss)
    with open(str(k)+"_TrainAcc.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(TrainAcc)
    with open(str(k)+"_TestLoss.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(TestLoss)
    with open(str(k)+"_TestAcc.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(TestAcc)
    with open(str(k)+"_FPR.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(All_FPR)
    with open(str(k)+"_TPR.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(All_TPR)
    with open(str(k)+"_Thresholds.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(All_thresholds)